# Import and Check OS

In [5]:
import sys


import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
import scipy.ndimage as nd

In [6]:
def change_contrast_brightness(image, alpha, beta):
    """
    https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html
    """
    new_image = np.zeros(image.shape, image.dtype)
    
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)
    return new_image

def gamma_adjustment(img_original, gamma):
    """
    https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html
    """
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    res = cv2.LUT(img_original, lookUpTable)
    return res

In [7]:
if 'win32' in sys.platform:
    ex_filepath = 'F:\Colonies_Data\data\plate 2_24.png'
if 'darwin' in sys.platform:
    ex_filepath = "/Volumes/Samsung_T5/Colonies_Data/data/plate 2_24.png"

# Find the plate

## Find position and radius of the plate

In [8]:
# https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
ex_filepath = "/Volumes/Samsung_T5/Segmentation_Data/VC0395_300mOsm_12.JPG"
IMG_SIZE = 1000

image = cv2.imread(ex_filepath)
image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
output = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, minDist=int(IMG_SIZE/3), dp=2, minRadius=int(IMG_SIZE/3), maxRadius=int(IMG_SIZE/2))
 
    
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circle_ = np.round(circles[0, :]).astype("int")
    print(circle_)
    # loop over the (x, y) coordinates and radius of the circles
    x, y, r = circle_[0]


[[525 461 470]]


## Extract the plate

In [9]:
img = Image.open(ex_filepath).convert("RGB").resize((1000,1000))

npImage=np.array(img)
image = change_contrast_brightness(npImage, .8, 10)

# Create same size alpha layer with circle
alpha = Image.new('L', img.size,0)
draw = ImageDraw.Draw(alpha)
draw.pieslice([x-r, y-r, x+r, y+r],0,360,fill=255)

# Convert alpha Image to numpy array
npAlpha=np.array(alpha)

# Add alpha layer to RGB
npImage=np.dstack((image,npAlpha))

# Save with alpha
Image.fromarray(npImage).save('result.png')

# Find Small circles

In [10]:
image = cv2.imread('result.png', cv2.IMREAD_UNCHANGED)
image2 = cv2.bitwise_not(image[:,:,:3])
output = image2.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, minDist=1, dp=2, minRadius=10, maxRadius=25, param2=37)
 
    
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circle_ = np.round(circles[0, :]).astype("int")
    print(circle_)
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circle_:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
 
    # show the output image
#     cv2.imshow("output", np.hstack([image2, output]))
#     cv2.waitKey(0)

[[705 933  24]
 [703 935  24]
 [699 937  24]
 ...
 [199 771  11]
 [671 941  11]
 [413 971  11]]


In [11]:
def window_stdev(arr, radius):
    c1 = nd.filters.uniform_filter(arr, radius*2, mode='constant', origin=-radius)
    c2 = nd.filters.uniform_filter(arr*arr, radius*2, mode='constant', origin=-radius)
    return ((c2 - c1*c1)**.5)[:-radius*2+1,:-radius*2+1]

## Attempting Template Matching

In [13]:
img = cv2.imread(ex_filepath,0)
img2 = img.copy()
template = cv2.imread("/Volumes/Samsung_T5/Segmentation_Data/VC0395_300mOsm_12_colony.JPG",0)
img_rgb = cv2.imread(ex_filepath)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.6
loc = np.where( res >= threshold)
print(loc)
for pt in zip(*loc[::-1]):
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

cv2.imwrite('res.png',img_rgb)

(array([ 898,  898,  898, ..., 2943, 2943, 2943]), array([2023, 2024, 2025, ..., 1715, 1716, 1717]))


True

In [49]:
matches = list(zip(*loc[::-1]))
matches = sorted(matches, key=lambda x:x[0])

In [50]:
unique_locs = []
current_loc = matches[0]
curr_lim = matches[0]
for match in matches:
    if ((match[0] < current_loc[0] + w) 
        and(match[1] < current_loc[1] + h)
        and(match[0] >= current_loc[0])
        and((match[1] >= current_loc[1] - h) and (match[1] <= current_loc[1] + h))): # Inside curr_loc's box
            current_lim = match
    else:
        unique_locs.append((current_loc, current_lim))
        current_loc = match
        curr_lim = match

In [51]:
unique_locs2 = sorted(unique_locs, key=lambda x:x[1][1])
unique_locs2

[((2021, 900), (2032, 908)),
 ((1282, 1320), (1296, 1323)),
 ((1718, 1439), (1718, 1440)),
 ((1662, 1444), (1674, 1447)),
 ((711, 1539), (719, 1542)),
 ((1394, 1548), (1416, 1549)),
 ((1791, 1854), (1810, 1856)),
 ((785, 2173), (803, 2174)),
 ((983, 2705), (996, 2715)),
 ((958, 2797), (982, 2721)),
 ((1718, 2927), (1723, 2937)),
 ((1701, 2929), (1716, 2943)),
 ((1717, 1440), (1716, 2943)),
 ((1717, 2927), (1717, 2943))]

In [52]:
unique_locs

[((711, 1539), (719, 1542)),
 ((785, 2173), (803, 2174)),
 ((958, 2797), (982, 2721)),
 ((983, 2705), (996, 2715)),
 ((1282, 1320), (1296, 1323)),
 ((1394, 1548), (1416, 1549)),
 ((1662, 1444), (1674, 1447)),
 ((1701, 2929), (1716, 2943)),
 ((1717, 1440), (1716, 2943)),
 ((1717, 2927), (1717, 2943)),
 ((1718, 1439), (1718, 1440)),
 ((1718, 2927), (1723, 2937)),
 ((1791, 1854), (1810, 1856)),
 ((2021, 900), (2032, 908))]

In [58]:
def is_close(loc1, loc2, w, h):
    loc1_f = np.array(loc1[1])
    loc2_f = np.array(loc2[1])
    diag_of_rect = np.sqrt(w ** 2 + h ** 2)
    if np.linalg.norm(loc1_f - loc2_f, 2) < max(2 * w, 2 * h):
        return True
    else:
        return False

In [59]:
true_unique_locs = unique_locs.copy()
for location1 in true_unique_locs:
    for location2 in true_unique_locs:
        if (location1 != location2) and is_close(location1, location2, w, h):
            true_unique_locs.remove(location2)

In [60]:
true_unique_locs

[((711, 1539), (719, 1542)),
 ((785, 2173), (803, 2174)),
 ((958, 2797), (982, 2721)),
 ((1282, 1320), (1296, 1323)),
 ((1394, 1548), (1416, 1549)),
 ((1662, 1444), (1674, 1447)),
 ((1717, 2927), (1717, 2943)),
 ((1791, 1854), (1810, 1856)),
 ((2021, 900), (2032, 908))]

In [61]:
img = cv2.imread(ex_filepath,0)
img2 = img.copy()
template = cv2.imread("/Volumes/Samsung_T5/Segmentation_Data/VC0395_300mOsm_12_colony.JPG",0)
img_rgb = cv2.imread(ex_filepath)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.6
loc = np.where( res >= threshold)
print(loc)
for pt in zip(*loc[::-1]):
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

cv2.imwrite('res.png',img_rgb)

(array([ 898,  898,  898, ..., 2943, 2943, 2943]), array([2023, 2024, 2025, ..., 1715, 1716, 1717]))


True

In [62]:
for ul in unique_locs:
    print(ul)

((711, 1539), (719, 1542))
((785, 2173), (803, 2174))
((958, 2797), (982, 2721))
((983, 2705), (996, 2715))
((1282, 1320), (1296, 1323))
((1394, 1548), (1416, 1549))
((1662, 1444), (1674, 1447))
((1701, 2929), (1716, 2943))
((1717, 1440), (1716, 2943))
((1717, 2927), (1717, 2943))
((1718, 1439), (1718, 1440))
((1718, 2927), (1723, 2937))
((1791, 1854), (1810, 1856))
((2021, 900), (2032, 908))


In [88]:
def find_max_boxes(locs, w, h):
    loc_1 = locs[0]
    loc_2 = locs[1]
    upper_left = (min(loc_1[0], loc_2[0]), max(loc_1[1], loc_2[1]))
    lower_right = (max(loc_1[0], loc_2[0]) + w, max(loc_1[1], loc_2[1]) + h)
    return (upper_left, lower_right)

def overlap(locs, w, h):
    loc_1 = locs[0]
    loc_2 = locs[1]
    if     
        return True
    else:
        return False

In [66]:
template = cv2.imread("/Volumes/Samsung_T5/Segmentation_Data/VC0395_300mOsm_12_colony.JPG",0)
img_rgb = cv2.imread(ex_filepath)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.55
loc = np.where( res >= threshold)
for ul in unique_locs:
    upper_left, lower_right = find_max_boxes(ul, w, h)
#     cv2.rectangle(img_rgb, ul[0], (ul[1][0] + w, ul[1][1] + h), (0,0,255), 2)
    cv2.rectangle(img_rgb, upper_left, lower_right, (0,255,0), 2)

cv2.imwrite('res.png',img_rgb)



True

In [67]:
unique_locs

[((711, 1539), (719, 1542)),
 ((785, 2173), (803, 2174)),
 ((958, 2797), (982, 2721)),
 ((983, 2705), (996, 2715)),
 ((1282, 1320), (1296, 1323)),
 ((1394, 1548), (1416, 1549)),
 ((1662, 1444), (1674, 1447)),
 ((1701, 2929), (1716, 2943)),
 ((1717, 1440), (1716, 2943)),
 ((1717, 2927), (1717, 2943)),
 ((1718, 1439), (1718, 1440)),
 ((1718, 2927), (1723, 2937)),
 ((1791, 1854), (1810, 1856)),
 ((2021, 900), (2032, 908))]

### Trying to merge close colony boxes

In [116]:

total_locs = unique_locs.copy()
len(total_locs)
locs_to_remove = []
all_locs = []
for ul_1 in total_locs:
    current_close = 0
    for ul_2 in total_locs:
        if is_close(ul_1, ul_2, w, h) and ul_1 != ul_2:
            all_locs.append(find_max_boxes((ul_1[0], ul_2[0]), w, h))
            locs_to_remove.append(ul_1)
            locs_to_remove.append(ul_2)
            current_close += 1
    if not current_close:
        all_locs.append(find_max_boxes(ul_1, w, h))

In [117]:
all_locs = set(all_locs)
all_locs

{((711, 1542), (820, 1633)),
 ((785, 2174), (904, 2265)),
 ((958, 2797), (1084, 2888)),
 ((1282, 1323), (1397, 1414)),
 ((1394, 1549), (1517, 1640)),
 ((1662, 1444), (1819, 1535)),
 ((1701, 2929), (1818, 3020)),
 ((1701, 2929), (1819, 3020)),
 ((1717, 2927), (1818, 3018)),
 ((1717, 2927), (1819, 3018)),
 ((1791, 1856), (1911, 1947)),
 ((2021, 908), (2133, 999))}

In [118]:
template = cv2.imread("/Volumes/Samsung_T5/Segmentation_Data/VC0395_300mOsm_12_colony.JPG",0)
img_rgb = cv2.imread(ex_filepath)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.55
loc = np.where( res >= threshold)
for ul in all_locs:
    upper_left, lower_right = ul[0], ul[1]
#     cv2.rectangle(img_rgb, ul[0], (ul[1][0] + w, ul[1][1] + h), (0,0,255), 2)
    cv2.rectangle(img_rgb, upper_left, lower_right, (0,255,0), 2)

cv2.imwrite('res2.png',img_rgb)

True